In [12]:
# import zipfile
# import os

# def unzip_archive(zip_path, extract_to='.'):
#     """
#     Unzips the given archive to the specified directory.

#     Parameters:
#     zip_path (str): The path to the zip file.
#     extract_to (str): The directory to extract the files to. Default is the current directory.
#     """
#     try:
#         with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#             zip_ref.extractall(extract_to)
#             print(f"Successfully extracted all files to {extract_to}")
#     except zipfile.BadZipFile:
#         print("Error: The file is not a zip file or it is corrupted.")
#     except FileNotFoundError:
#         print("Error: The file was not found.")
#     except Exception as e:
#         print(f"An unexpected error occurred: {e}")

# # Path to the zip file
# zip_path = 'train_data.zip'  # replace with the actual path to your zip file
# # Directory to extract files to
# extract_to = '/'  # replace with your desired extraction directory

# # Ensure the extraction directory exists
# os.makedirs(extract_to, exist_ok=True)

# # Unzip the archive
# unzip_archive(zip_path, extract_to)

Error: The file is not a zip file or it is corrupted.


In [3]:
import pandas as pd
import pyarrow.parquet as pq

# Загрузка Parquet файла
file_path = '../data/test_data/test_data_0.pq'
df = pd.read_parquet(file_path)

# Группировка данных по пользователю
user_groups = df.groupby('id')

# Создание списка групп
grouped_data = [group for _, group in user_groups]

# Определение размера каждой части
total_groups = len(grouped_data)
part_size = total_groups // 12

# Разделение на 12 частей, избегая разбиения истории одного пользователя
parts = []
for i in range(12):
    start_index = i * part_size
    end_index = (i + 1) * part_size if i < 11 else total_groups
    part = pd.concat(grouped_data[start_index:end_index])
    parts.append(part)

# Сохранение каждой части в отдельный Parquet файл
for i, part in enumerate(parts):
    part_file_path = f'part_{i+1}.pqt'
    part.to_parquet(part_file_path)

print("Разделение завершено. Файлы сохранены.")

Разделение завершено. Файлы сохранены.


### 0. Imports and requirements

* Для моделирования нам понадобится реализация градиентного бустинга из `lightgbm`. Для обработки данных &ndash; стандартный DS стек из `pandas`, `numpy` и `sklearn`. Так как мы будем использовать достаточно объемные выборки данных (кредитные истории клиентов), то нужно будет эффективно читать данные и выделять из них признаки, чтобы иметь возможность строить решение даже на локальных машинах с ограничениями по оперативной памяти.  Поэтому нам понадобятся инструменты для работы с форматом данных `Parquet`. Библиотека `pandas` представляет необходимый инструментарий (рекомендуем установить модуль `fastparquet`, это позволит еще быстрее считывать данные).

In [1]:
%load_ext autoreload
%autoreload 2

import gc
import os
import sys
import pandas as pd
import numpy as np
import tqdm
import seaborn as sns


import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score

import catboost as cb
import lightgbm as lgb

### 1. Reading Data

* В данном соревновании участникам предстоит работать с таким форматом данных, как `Parquet`. Узнать подробнее об этом формате Вы можете самостоятельно. Самое главное &ndash; это крайне эффективный бинарный формат сжатия данных по колонокам. Однако, для непосредственной работы с данными и построения моделей нам нужно прочитать их и трансформировать в pandas.DataFrame. При этом сделать это эффективно по памяти.

In [2]:
TRAIN_DATA_PATH = '/trinity/home/team08/workspace/data/train_data/'# "/content/data/train_data/"
TEST_DATA_PATH = "/trinity/home/team08/workspace/data/test_data/"

TRAIN_TARGET_PATH = "/trinity/home/team08/workspace/data/train_target.csv"

In [3]:
import os
import pandas as pd
import tqdm
from typing import List


def read_parquet_dataset_from_local(path_to_dataset: str, start_from: int = 0, num_parts_to_read: int = 2,
                                    columns: List[str] = None, verbose: bool = False) -> pd.DataFrame:
    """
    Читает ``num_parts_to_read`` партиций и преобразует их к pandas.DataFrame.

    Параметры:
    -----------
    path_to_dataset: str
        Путь до директории с партициями.
    start_from: int, default=0
        Номер партиции, с которой начать чтение.
    num_parts_to_read: int, default=2
        Число партиций, которые требуется прочитать.
    columns: List[str], default=None
        Список колонок, которые нужно прочитать из каждой партиции. Если None, то считываются все колонки.

    Возвращаемое значение:
    ----------------------
    frame: pandas.DataFrame
        Прочитанные партиции, преобразованные к pandas.DataFrame.
    """

    res = []
    start_from = max(0, start_from)
    # dictionory of format {partition number: partition filename}
    dataset_paths = {int(os.path.splitext(filename)[0].split("_")[-1]): os.path.join(path_to_dataset, filename)
                     for filename in os.listdir(path_to_dataset)}
    chunks = [dataset_paths[num] for num in sorted(dataset_paths.keys()) if num>=start_from][:num_parts_to_read]

    if verbose:
        print("Reading chunks:", *chunks, sep="\n")
    for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):
        chunk = pd.read_parquet(chunk_path, columns=columns)
        res.append(chunk)
    return pd.concat(res).reset_index(drop=True)

* Для примера прочитаем одну партицию в память и оценим, сколько RAM занимает весь DataFrame

In [4]:
# в какой папке мы находимся
import os
os.getcwd()

'/trinity/home/team08/workspace'

In [5]:
os.listdir()

['.checkpoints',
 'AlfaBattle2.0',
 'tmp',
 '.bashrc',
 'baselinegrad.ipynb',
 'gleb_pytorch_baseline.ipynb',
 '.nv',
 '.ipynb_checkpoints',
 '.keras',
 '.cache',
 'data_for_competition',
 '.config',
 'smiles-master',
 'utils.py',
 'artiom_preprocessing.ipynb',
 '.bash_history',
 'rnn_baseline',
 '.jupyter',
 'data',
 '.ipython',
 'main_project',
 'smiles_gb_baseline_done.ipynb',
 '.local',
 'checkpoints',
 '__pycache__']

In [13]:
data_frame = read_parquet_dataset_from_local(TRAIN_DATA_PATH, start_from=0, num_parts_to_read=6)

memory_usage_of_frame = data_frame.memory_usage(index=True).sum() / 10**9
expected_memory_usage = memory_usage_of_frame * 12
print(f"Объем памяти в RAM одной партиции данных с кредитными историями: {round(memory_usage_of_frame, 3)} GB")
print(f"Ожидаемый размер в RAM всего датасета: {round(expected_memory_usage, 3)} GB")

/tmp/ipykernel_2749022/2416791525.py:38: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):


Reading dataset with pandas:   0%|          | 0/6 [00:00<?, ?it/s]

Объем памяти в RAM одной партиции данных с кредитными историями: 6.095 GB
Ожидаемый размер в RAM всего датасета: 73.142 GB


* Итого, при чтении всех данных сразу, они займут значительный объем памяти. Решение &ndash; читать данные итеративно небольшими чанками. Чанки организованы таким образом, что для конкретного клиента вся информация о его кредитной истории до момента подачи заявки на кредит расположена внутри одного чанка. Это позволяет загружать данные в память небольшими порциями, выделять все необходимые признаки и получать результирующий фрейм для моделирования. Для этих целей мы предоставляем вам функцию `utils.read_parquet_dataset_from_local`.

Так же все чанки данных упорядочены по возрастанию даты заявки на кредит.

In [14]:
del data_frame
gc.collect()

224

### 2. Feature Extraction

* В рамках данного соревнования участникам предстоит работать с кредитной историей клиентов. На основе кредитной истории клиента до момента подачи заявки на новый кредит нужно оценить, насколько благонадежным является клиент, и определить вероятность его ухода в дефолт по новому кредиту. Каждый кредит описывается набором из 60 категориальных признаков.


* Базовым подходом к решению этой задачи является построение классических моделей машинного обучения на аггрегациях от последовательностей категориальных признаков. В данном случае мы закодируем признаки с помощью one-hot-encoding'а, применим к ним count-аггрегирование и обучим на этом градиентный бустинг из реализации `lightgbm`.
     
     
 * Описанный подход к обработке кредитных историй клиентов реализован в виде класс-трансформера `CountAggregator` ниже:

In [6]:
TRAIN_FEATURES_PATH = "/trinity/home/team08/workspace/data/train_features_gb/"
TEST_FEATURES_PATH =  "/trinity/home/team08/workspace/data/test_features_gb/"

In [23]:
! rm -r $TRAIN_FEATURES_PATH
! mkdir $TRAIN_FEATURES_PATH
! rm -r $TEST_FEATURES_PATH
! mkdir $TEST_FEATURES_PATH

In [8]:
class CountAggregator(object):

    def __init__(self):
        self.encoded_features = None

    def __extract_count_aggregations(self, data_frame: pd.DataFrame, mode: str) -> pd.DataFrame:
        # one-hot-encoding
        feature_columns = list(data_frame.columns.values)
        feature_columns.remove("id")
        # feature_columns.remove("rn")

        dummies = pd.get_dummies(data_frame[feature_columns], columns=feature_columns)
        dummy_features = dummies.columns.values

        ohe_features = pd.concat([data_frame, dummies], axis=1)
        ohe_features = ohe_features.drop(columns=feature_columns)

        # count aggregation
        ohe_features.groupby("id")
        features = ohe_features.groupby("id")[dummy_features].sum().reset_index(drop=False)
        return features

    def __transform_data(self, path_to_dataset: str, num_parts_to_preprocess_at_once: int = 1, num_parts_total: int=50,
                                     mode: str = "fit_transform", save_to_path=None, verbose: bool=False):
        assert mode in ["fit_transform", "transform"], f"Unrecognized mode: {mode}! Please use one of the following modes: \"fit_transform\", \"transform\""
        preprocessed_frames = []
        for step in tqdm.tqdm_notebook(range(0, num_parts_total, num_parts_to_preprocess_at_once), desc="Transforming sequential credit data"):
            data_frame = read_parquet_dataset_from_local(path_to_dataset, start_from=step,
                                                         num_parts_to_read=num_parts_to_preprocess_at_once,
                                                         verbose=verbose)
            features = self.__extract_count_aggregations(data_frame, mode=mode)
            if save_to_path:
                features.to_parquet(os.path.join(save_to_path, f"processed_chunk_{step}.pq"))
            preprocessed_frames.append(features)

        features = pd.concat(preprocessed_frames)
        features.fillna(np.uint8(0), inplace=True)
        dummy_features = list(features.columns.values)
        dummy_features.remove("id")
        if mode == "fit_transform":
            self.encoded_features = dummy_features
        else:
            assert not self.encoded_features is None, "Transformer not fitted"
            for col in self.encoded_features:
                if not col in dummy_features:
                    features[col] = np.uint8(0)
        return features[["id"]+self.encoded_features]

    def fit_transform(self, path_to_dataset: str, num_parts_to_preprocess_at_once: int = 1, num_parts_total: int = 50,
                      save_to_path=None, verbose: bool=False):
        return self.__transform_data(path_to_dataset=path_to_dataset,
                                     num_parts_to_preprocess_at_once=num_parts_to_preprocess_at_once,
                                     num_parts_total=num_parts_total, mode="fit_transform",
                                     save_to_path=save_to_path, verbose=verbose)
    def transform(self, path_to_dataset: str, num_parts_to_preprocess_at_once: int = 1, num_parts_total: int=50,
                  save_to_path=None, verbose: bool=False):
        return self.__transform_data(path_to_dataset=path_to_dataset,
                                     num_parts_to_preprocess_at_once=num_parts_to_preprocess_at_once,
                                     num_parts_total=num_parts_total, mode="transform",
                                     save_to_path=save_to_path, verbose=verbose)

In [9]:
%%time
aggregator = CountAggregator()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.48 µs


In [10]:
train_data = aggregator.fit_transform(TRAIN_DATA_PATH, num_parts_to_preprocess_at_once=1, num_parts_total=1,
                                      save_to_path=TRAIN_FEATURES_PATH, verbose=True)

/tmp/ipykernel_3704710/1845575573.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for step in tqdm.tqdm_notebook(range(0, num_parts_total, num_parts_to_preprocess_at_once), desc="Transforming sequential credit data"):


Transforming sequential credit data:   0%|          | 0/1 [00:00<?, ?it/s]

Reading chunks:
/trinity/home/team08/workspace/data/train_data/train_data_0.pq


/tmp/ipykernel_3704710/2416791525.py:38: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):


Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
train_data.isna().sum().sum()

0

In [12]:
print(f"Объем оперативной памяти, занимаемой датафреймом с признаками: {train_data.memory_usage(index=True).sum() / 1e9}")

Объем оперативной памяти, занимаемой датафреймом с признаками: 0.894000128


* Также извлечем признаки для заявок из тестовой выборки

In [13]:
test_data = aggregator.transform(TEST_DATA_PATH, num_parts_to_preprocess_at_once=1, num_parts_total=1,
                                 save_to_path=TEST_FEATURES_PATH, verbose=True)

/tmp/ipykernel_3704710/1845575573.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for step in tqdm.tqdm_notebook(range(0, num_parts_total, num_parts_to_preprocess_at_once), desc="Transforming sequential credit data"):


Transforming sequential credit data:   0%|          | 0/1 [00:00<?, ?it/s]

Reading chunks:
/trinity/home/team08/workspace/data/test_data/test_data_0.pq


/tmp/ipykernel_3704710/2416791525.py:38: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):


Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

# Preprocessing

In [22]:
train_data

,id,rn_1,rn_2,rn_3,rn_4,rn_5,rn_6,rn_7,rn_8,rn_9,...,enc_loans_credit_type_4,enc_loans_credit_type_5,enc_loans_account_cur_0,enc_loans_account_cur_1,enc_loans_account_cur_2,enc_loans_account_cur_3,pclose_flag_0,pclose_flag_1,fclose_flag_0,fclose_flag_1
0,0,1,1,1,1,1,1,1,1,1,...,7,0,0,10,0,0,9,1,8,2
1,1,1,1,1,1,1,1,1,1,1,...,8,0,0,14,0,0,13,1,12,2
2,2,1,1,1,0,0,0,0,0,0,...,1,0,0,3,0,0,1,2,1,2
3,3,1,1,1,1,1,1,1,1,1,...,9,1,0,15,0,0,10,5,9,6
4,4,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,249995,1,1,1,1,1,1,1,1,1,...,7,3,0,13,0,0,12,1,10,3
249996,249996,1,1,1,1,1,1,1,1,1,...,9,3,0,23,0,0,18,5,18,5
249997,249997,1,1,1,1,1,1,1,0,0,...,2,0,0,6,1,0,6,1,5,2
249998,249998,1,1,1,1,1,0,0,0,0,...,2,0,0,5,0,0,3,2,2,3


## PCA

In [41]:
# len(train_data.columns)
train_data

array([[-1.24999500e+05,  1.44269448e+01,  2.11476527e+00, ...,
         3.61258371e-03,  1.39142835e-03, -2.00125881e-03],
       [-1.24998500e+05,  3.57228616e+01,  1.55654511e+00, ...,
        -2.32150468e-02, -5.31921128e-02,  4.59394416e-01],
       [-1.24997500e+05, -3.04035398e+01, -3.25339730e+00, ...,
        -4.35115421e-03,  2.06637142e-01,  9.49395513e-03],
       ...,
       [ 1.24997500e+05, -8.09664657e+00,  1.15490180e+01, ...,
        -2.02929130e-03, -4.78833873e-03,  3.45622812e-03],
       [ 1.24998500e+05, -2.00633918e+01, -7.83170743e+00, ...,
        -1.06659888e-02, -2.62824772e-02, -4.30602296e-01],
       [ 1.24999500e+05, -2.90853913e+01, -3.33234036e+00, ...,
        -2.38750889e-03, -3.34308559e-03,  8.89567620e-04]])

In [50]:
train_data_original[list(set(train_data_original.columns) - set('id'))]

,pre_maxover2limit_7,enc_paym_8_3,pre_pterm_5,pre_fterm_2,pre_loans3060_8,is_zero_util_0,pre_maxover2limit_17,pre_loans_credit_limit_11,pre_maxover2limit_9,is_zero_over2limit_0,...,pre_loans_credit_limit_2,enc_paym_13_1,enc_paym_12_1,pre_util_0,pre_pterm_8,pre_since_confirmed_3,pre_loans5_5,enc_paym_7_2,enc_paym_18_3,pre_till_pclose_12
0,0,5,0,2,0,4,9,2,0,1,...,1,0,0,0,0,0,0,0,7,4
1,0,7,0,1,0,4,12,1,0,2,...,0,0,0,1,1,2,0,1,10,0
2,0,1,0,1,0,2,2,0,0,0,...,0,1,1,0,0,0,0,0,2,0
3,0,2,0,1,0,7,14,3,0,1,...,0,0,0,0,1,1,0,0,8,0
4,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,0,5,0,2,0,4,12,0,0,0,...,0,0,0,0,0,0,0,0,13,0
249996,0,12,0,0,0,5,20,2,0,3,...,0,1,1,0,5,4,0,0,20,0
249997,0,1,0,0,0,0,7,0,0,0,...,3,0,0,0,0,0,0,0,3,1
249998,0,3,0,0,0,4,3,0,0,2,...,0,0,0,0,0,0,0,0,5,0


In [51]:
from sklearn.decomposition import PCA
pca = PCA(n_components=300)
train_data_pca_300 = pca.fit_transform(train_data_original[list(set(train_data_original.columns) - set('id'))])
train_data_pca_300 = pd.DataFrame(train_data_pca_300)
train_data_pca_300['id'] = train_data_original.id
train_data = train_data_pca_300
train_data

array([[-1.24999500e+05,  1.44269448e+01,  2.11476532e+00, ...,
         3.61332244e-03,  1.40217197e-03, -2.53712226e-03],
       [-1.24998500e+05,  3.57228616e+01,  1.55654510e+00, ...,
        -2.34448615e-02, -5.42567246e-02,  4.50180155e-01],
       [-1.24997500e+05, -3.04035398e+01, -3.25339730e+00, ...,
        -4.49617150e-03,  2.06834258e-01,  9.19693564e-03],
       ...,
       [ 1.24997500e+05, -8.09664657e+00,  1.15490181e+01, ...,
        -2.02521379e-03, -4.79503816e-03,  3.12386209e-03],
       [ 1.24998500e+05, -2.00633918e+01, -7.83170746e+00, ...,
        -1.05331943e-02, -2.58500581e-02, -4.29499683e-01],
       [ 1.24999500e+05, -2.90853913e+01, -3.33234039e+00, ...,
        -2.38480729e-03, -3.34775176e-03,  1.09899281e-03]])

In [66]:
pca_100 = PCA(n_components=100)
train_data_pca_100 = pca_100.fit_transform(train_data_original[list(set(train_data_original.columns) - set('id'))])
train_data_pca_100 = pd.DataFrame(train_data_pca_100)
train_data_pca_100['id'] = train_data_original.id
train_data = train_data_pca_100
train_data

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,id
0,-124999.499872,14.426945,2.114765,-1.824454,2.043632,-5.218412,0.859625,-2.601138,1.021671,4.836605,...,-0.314624,-0.019337,0.062554,-0.372865,0.581824,-1.651369,0.469341,-0.009452,0.571905,0
1,-124998.499686,35.722862,1.556545,0.422477,4.025992,4.160893,2.216237,4.930542,1.009836,0.909831,...,-0.482562,0.153194,-0.247915,-0.636634,-0.355252,-0.292392,1.851764,-0.253416,0.873255,1
2,-124997.500278,-30.403540,-3.253397,-0.369454,2.214616,-1.108131,-2.756194,-0.524282,-0.647567,-1.024185,...,0.117704,0.083604,-0.164934,-0.285821,-0.454955,0.012468,-0.334163,-0.061103,-0.330581,2
3,-124996.499629,41.503185,22.703125,3.919561,4.814531,-11.707874,-2.562757,-2.139824,-2.040326,1.059640,...,-0.499839,0.202336,-1.393393,0.578597,0.994765,-1.047867,0.175675,-0.386790,-1.606105,3
4,-124995.500372,-40.442081,-5.109868,1.294659,0.517693,0.429441,0.670954,-0.895549,-1.082751,-1.208074,...,-0.042891,-0.063195,-0.066573,-0.083902,0.071273,0.057586,-0.006697,0.064270,-0.048778,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,124995.500358,36.144503,-5.836554,-15.193726,4.830190,-5.616022,-0.113028,3.518435,-3.004229,1.001524,...,-0.898713,0.788757,0.073983,0.545973,0.036581,0.489406,-0.184741,-0.505169,-0.088941,249995
249996,124996.500880,93.489786,-9.444995,-3.305539,6.544111,-0.541930,-1.404791,4.457787,0.786336,-7.229550,...,0.168721,-0.732330,-0.528880,0.906862,0.735073,-0.828644,-0.040955,-0.119043,-0.676026,249996
249997,124997.499916,-8.096647,11.549018,2.885977,-2.437146,1.141448,1.144147,-3.959393,-0.218366,-0.360518,...,0.035829,1.369156,-0.505115,0.511071,-0.690950,-0.598449,-0.270070,0.679262,0.098850,249997
249998,124998.499835,-20.063392,-7.831707,-0.390510,2.915841,-3.621065,-4.044264,2.634357,-0.499886,-1.213386,...,0.262570,0.293860,-0.195527,-0.723548,-0.883042,-0.008474,0.137605,0.152444,1.393133,249998


## feature_selection: mutual_info_classif

In [74]:
train_data_target_original = train_target.merge(train_data_original, on="id")
train_data_target_original

,id,flag,rn_1,rn_2,rn_3,rn_4,rn_5,rn_6,rn_7,rn_8,...,enc_loans_credit_type_4,enc_loans_credit_type_5,enc_loans_account_cur_0,enc_loans_account_cur_1,enc_loans_account_cur_2,enc_loans_account_cur_3,pclose_flag_0,pclose_flag_1,fclose_flag_0,fclose_flag_1
0,0,0,1,1,1,1,1,1,1,1,...,7,0,0,10,0,0,9,1,8,2
1,1,0,1,1,1,1,1,1,1,1,...,8,0,0,14,0,0,13,1,12,2
2,2,0,1,1,1,0,0,0,0,0,...,1,0,0,3,0,0,1,2,1,2
3,3,0,1,1,1,1,1,1,1,1,...,9,1,0,15,0,0,10,5,9,6
4,4,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,249995,0,1,1,1,1,1,1,1,1,...,7,3,0,13,0,0,12,1,10,3
249996,249996,0,1,1,1,1,1,1,1,1,...,9,3,0,23,0,0,18,5,18,5
249997,249997,0,1,1,1,1,1,1,1,0,...,2,0,0,6,1,0,6,1,5,2
249998,249998,0,1,1,1,1,1,0,0,0,...,2,0,0,5,0,0,3,2,2,3


In [82]:
# from sklearn.feature_selection import mutual_info_classif
# import matplotlib.pyplot as plt
# feature_cols_current = list(set(train_data_target_original.columns) - set(['id', 'flag']))
# X = train_data_target_original[feature_cols_current]
# y = train_data_target_original.flag
# importances = mutual_info_classif(X, y)
# feature_importances = pd.Series(importances, df.columns[0:len(df.columns)])
# feature_importances.plot(kind='barh', color='teal')
# plt.show()

## Критерий хи-квадрат (Chi-square Test)

In [78]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
chi2_features = SelectKBest(chi2, k=100)
X_kbest_features_chi2 = chi2_features.fit_transform(X, y)

print("Количество признаков до преобразования:", X.shape[1])
print("Количество признаков после преобразования:", X_kbest_features_chi2.shape[1])

train_data_chi2_100 = pd.DataFrame(X_kbest_features_chi2)
train_data_chi2_100['id'] = train_data_original.id
train_data = train_data_chi2_100
train_data

Количество признаков до преобразования: 446
Количество признаков после преобразования: 100


,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,id
0,9,0,3,2,2,3,0,6,0,3,...,0,0,5,10,0,0,0,1,1,0
1,12,0,10,1,4,7,1,10,0,4,...,0,1,7,12,1,0,2,1,0,1
2,2,0,1,0,0,1,1,1,0,0,...,0,1,2,2,0,0,0,0,0,2
3,14,0,4,0,7,2,1,8,1,7,...,0,1,13,15,3,0,0,0,0,3
4,1,0,0,0,0,1,0,1,0,0,...,0,1,0,1,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,12,0,11,0,0,2,2,9,0,1,...,0,0,8,13,0,1,0,0,0,249995
249996,20,0,14,1,3,11,1,18,1,5,...,3,1,11,23,1,1,1,1,0,249996
249997,7,0,0,1,4,1,0,7,0,4,...,0,0,6,7,0,0,4,0,3,249997
249998,3,0,3,0,0,3,0,1,0,0,...,0,0,2,4,0,0,0,0,0,249998


## Критерий Фишера (F-тест)

In [ ]:
# from skfeature.function.similarity_based import fisher_score
# import matplotlib.pyplot as plt

# # Вычисляем критерий
# # Где X, y - входные и выходные данные соответственно.
# # ranks = fisher_score.fisher_score(X.to_numpy(), y) ## ERROR
# ranks = fisher_score.fisher_score(X.to_numpy(), y.to_numpy())
# # ПАМЯТЬ исчерпана
# # Делаем график наших "фич"
# # Где df - ваш датасет
# feature_importances = pd.Series(ranks, train_data_target_original.columns)
# feature_importances.plot(kind='barh', color='teal')
# plt.show()

## Прямой отбор признаков: SequentialFeatureSelector

In [1]:
from sklearn.linear_model import LogisticRegression
from mlxtend.feature_selection import SequentialFeatureSelector

lr = LogisticRegression(class_weight = 'balanced', solver = 'lbfgs', random_state=42, n_jobs=-1, max_iter=50)
ffs = SequentialFeatureSelector(lr, k_features='best', forward = True, n_jobs=-1)

ffs.fit(X, y)

features = list(ffs.k_feature_names_)
features = list(map(int, features))

y_pred = lr.predict(X_train[features])
features

NameError: name 'X' is not defined

**Исчерпывающий выбор признаков**

Это самый надежный метод выбора функций из всех существующих. Его суть – оценка каждого подмножества функций методом перебора. Это означает, что метод пропускает все возможные комбинации переменных через алгоритм и возвращает наиболее эффективное подмножество.

In [ ]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector
from sklearn.ensemble import RandomForestClassifier

# создаем ExhaustiveFeatureSeLlector объект.
efs = ExhaustiveFeatureSelector(RandomForestClassifier(),
        min_features=4,
        max_features=8,
        scoring='roc_auc',
        cv=2)

efs = efs.fit(X, Y)

# выводим выбранные признаки
selected_features = X_train.columns[list(efs.best_idx_)]
print(selected_features)

# выводим финальную оценку прогнозирования.
print(efs.best_score_)

**Рекурсивное исключение признаков**

Сначала модель обучается на начальной выборке признаков, и важность каждой функции определяется либо с помощью атрибута coef_ или feature_importances_. Затем наименее важные “фичи” удаляются из текущего набора. Процедура рекурсивно повторяется для сокращенного набора до тех пор, пока в конечном итоге не будет достигнуто желаемое количество признаков для выбора.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

lr = LogisticRegression(class_weight = 'balanced', solver = 'lbfgs', random_state=42, n_jobs=-1, max_iter=50e)

rfe = RFE(lr, n_features_to_select=7)
rfe.fit(X_train, y_train)
# X_train, y_train - входные и выходные данные с обучающей выборки соответственно.
y_pred = rfe.predict(X_train)

#### 2.3. Встроенные методы (embedded methods)
Эти методы включают в себя преимущества первых двух, а также уменьшают вычислительные затраты. Отличительной особенностью встроенных методов является извлечение “фич” на этапе конкретной итерации.

**Регуляризация LASSO (L1)**

Регуляризация состоит в добавлении штрафа (penalty) к различным параметрам модели во избежание чрезмерной подгонки. При регуляризации линейной модели штраф применяется к коэффициентам, умножающим каждый из предикторов. Lasso-регуляризация обладает свойством, позволяющим уменьшить некоторые коэффициенты до нуля. Следовательно, такие “фичи” можно будет просто удалить из модели.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

# Устанавливаем наш параметр регуляризации C=1
logistic = LogisticRegression(C=1, penalty="l1", solver='liblinear', random_state=7).fit(X, y)
# Где X, y - входные и выходные данные соответственно.
model = SelectFromModel(logistic, prefit=True)

X_new = model.transform(X)

# Выбираем нужные нам столбцы из датасета без нулевых данных
# Где “selected_features” - предварительно выбранные нами признаки (см. по предыдущим методам)
selected_columns = selected_features.columns[selected_features.var() != 0]
print(selected_columns)

**Метод с использованием Случайного Леса (Random Forest Importance)**

Стратегии на основе дерева, используемые случайными лесами, естественным образом ранжируются по тому, насколько хорошо они улучшают чистоту модели в плане данных. Таким образом, “обрезая” деревья ниже определенного коэффициента, мы можем подобрать наиболее важные признаки.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# создаем случайное дерево с вашими гипер параметрами
model = RandomForestClassifier(n_estimators=340)

# Обучаем модель на вашей выборке; Где X, y - входные и выходные данные соответственно.
model.fit(X, y)

# Подбираем самые важные признаки
importances = model.feature_importances_

# Создаем отдельный датасет для визуализации
final_df = pd.DataFrame({"Features" : pd.DataFrame(X).columns, "Importances" : importances})
final_df.set_index('Importances')

# Сортируем их по возрастанию для лучшей визуализации
final_df = final_df.sort_values('Importances')

# Выводим на график
final_df.plot.bar(color = 'teal')

## Последовательный отбор признаков: SequentialFeatureSelector

In [ ]:
from sklearn.linear_model import LogisticRegression
from mlxtend.feature_selection import SequentialFeatureSelector

lr_s = LogisticRegression(class_weight = 'balanced', solver = 'lbfgs', random_state=42, n_jobs=-1, max_iter=50)
lr_s.fit(X, y)

bfs = SequentialFeatureSelector(lr_s, k_features='best', forward = False, n_jobs=-1)
bfs.fit(X, y)
features = list(bfs.k_feature_names_)
features = list(map(int, features))
lr_s.fit(X_train[features], y_train)
y_pred = lr_s.predict(x_train[features])

### 3. Train Model: LightGBM + CV

* Для обучения модели помимо признаковых описаний нам также нужна целевая переменная. Целевая переменная для тренировочной выборки содержится в файле train_target.csv.

In [14]:
TRAIN_TARGET_PATH = "/trinity/home/team08/workspace/data/train_target.csv"

In [35]:
train_target = pd.read_csv(TRAIN_TARGET_PATH)
train_target

,id,flag
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
2999995,2999995,0
2999996,2999996,0
2999997,2999997,0
2999998,2999998,0


In [37]:
train_data_target

,id,flag,rn_1,rn_2,rn_3,rn_4,rn_5,rn_6,rn_7,rn_8,...,enc_loans_credit_type_4,enc_loans_credit_type_5,enc_loans_account_cur_0,enc_loans_account_cur_1,enc_loans_account_cur_2,enc_loans_account_cur_3,pclose_flag_0,pclose_flag_1,fclose_flag_0,fclose_flag_1
0,0,0,1,1,1,1,1,1,1,1,...,7,0,0,10,0,0,9,1,8,2
1,1,0,1,1,1,1,1,1,1,1,...,8,0,0,14,0,0,13,1,12,2
2,2,0,1,1,1,0,0,0,0,0,...,1,0,0,3,0,0,1,2,1,2
3,3,0,1,1,1,1,1,1,1,1,...,9,1,0,15,0,0,10,5,9,6
4,4,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,249995,0,1,1,1,1,1,1,1,1,...,7,3,0,13,0,0,12,1,10,3
249996,249996,0,1,1,1,1,1,1,1,1,...,9,3,0,23,0,0,18,5,18,5
249997,249997,0,1,1,1,1,1,1,1,0,...,2,0,0,6,1,0,6,1,5,2
249998,249998,0,1,1,1,1,1,0,0,0,...,2,0,0,5,0,0,3,2,2,3


In [79]:
train_data_target = train_target.merge(train_data, on="id")
targets = train_data_target["flag"].values

cv = KFold(n_splits=5, random_state=100, shuffle=True)

oof = np.zeros(len(train_data_target))
train_preds = np.zeros(len(train_data_target))

models = []

tree_params = {
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.05,
    "max_depth": 5,
    "reg_lambda": 1,
    "num_leaves": 64,
    "n_jobs": 5,
    "n_estimators": 2000
}

for fold_, (train_idx, val_idx) in enumerate(cv.split(train_data_target, targets), 1):
    print(f"Training with fold {fold_} started")
    lgb_model = lgb.LGBMClassifier(**tree_params)
    train, val = train_data_target.iloc[train_idx], train_data_target.iloc[val_idx]

    feature_cols_current = list(set(train.columns) - set(['id', 'flag']))
    lgb_model.fit(train[feature_cols_current], train.flag.values, eval_set=[(val[feature_cols_current], val.flag.values)],
              early_stopping_rounds=50, verbose=50)

    oof[val_idx] = lgb_model.predict_proba(val[feature_cols_current])[:, 1]
    train_preds[train_idx] += lgb_model.predict_proba(train[feature_cols_current])[:, 1] / (cv.n_splits-1)
    models.append(lgb_model)
    print(f"Training with fold {fold_} completed")

Training with fold 1 started
Training until validation scores don't improve for 50 rounds
[50]	valid_0's auc: 0.751423
[100]	valid_0's auc: 0.754032
[150]	valid_0's auc: 0.754343
[200]	valid_0's auc: 0.754316
Early stopping, best iteration is:
[180]	valid_0's auc: 0.754747
Training with fold 1 completed
Training with fold 2 started
Training until validation scores don't improve for 50 rounds
[50]	valid_0's auc: 0.74914
[100]	valid_0's auc: 0.754184
[150]	valid_0's auc: 0.755192
[200]	valid_0's auc: 0.75488
Early stopping, best iteration is:
[171]	valid_0's auc: 0.75544
Training with fold 2 completed
Training with fold 3 started
Training until validation scores don't improve for 50 rounds
[50]	valid_0's auc: 0.764949
[100]	valid_0's auc: 0.767003
[150]	valid_0's auc: 0.767287
[200]	valid_0's auc: 0.767434
Early stopping, best iteration is:
[164]	valid_0's auc: 0.76756
Training with fold 3 completed
Training with fold 4 started
Training until validation scores don't improve for 50 rounds

In [80]:
print("Train roc-auc: ", roc_auc_score(targets, train_preds)) 

Train roc-auc:  0.8108490403418046


In [81]:
print("CV roc-auc: ", roc_auc_score(targets, oof))

CV roc-auc:  0.7608866729699238


# Results


In [23]:
results = {
    'Vanila': {'Train roc-auc': 0.8531, 'CV roc-auc': 0.7699},
    'PCA300': {'Train roc-auc': 0.9041, 'CV roc-auc': 0.7583},
    'PCA100': {'Train roc-auc': 0.8690, 'CV roc-auc': 0.7464},
    'chi2_100': {'Train roc-auc': 0.8108, 'CV roc-auc': 0.7608}
}